In [139]:
import json
import os
from bs4 import BeautifulSoup, Tag

In [140]:
folder_path = "data_raw"

In [141]:
html_files = [f for f in os.listdir("downloads") if f.endswith('.html')]

In [142]:
def find_last_h2(element):
    current_element = element
    while True:
        current_element = current_element.previous_element
        if current_element is None:
            return None
        if isinstance(current_element, Tag) and current_element.name in ['h2','h3'] and 'leg' not in current_element.text:
            return current_element

In [143]:
def find_next_fdate(element):
    current_element = element
    while True:
        current_element = current_element.next_element
        if current_element is None:
            return None
        if isinstance(current_element, Tag) and 'fdate' in current_element.get('class', []):
            return current_element.text.split("(")[0].split('[')[0].replace("\xa0"," ").strip()

In [144]:
def scrape_matches(filename):
    
    with open(os.path.join("downloads", filename), "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
    
    filename = filename.replace(".html","_2.json")
    
    print(f"scraping {filename}")

    competition = filename[:-7].replace("_"," ")
    
    zapasy = []
    
    fotbaltabulky = soup.find_all(class_='wikitable')
    
    for f in fotbaltabulky:

        phase = find_last_h2(f).text.split('[')[0].strip()
                
        if (phase != 'Teams') & (phase != 'Bracket') & ('Team 1' in f.text):
        
            for r in f.find_all('tr'):

                if r.find('td'):

                    try:

                        match1, match2 = {}, {}

                        match1['competition'], match2['competition'] = competition, competition
                        match1['phase'], match2['phase'] = phase, phase

                        match1['homeTeam'] = r.find_all('td')[0].text.strip()
                        match1['homeTeamCountry'] = r.find_all('td')[0].find('img')['alt'].strip()
                        match1['awayTeam'] = r.find_all('td')[2].text.strip()
                        
                        odkazy = r.find_all('a')
                        try:
                            for o in odkazy:
                                if match1['homeTeam'] in o.text:
                                    match1['homeTeamUrl'] = o['href'].replace('/wiki/','')
                                if match1['awayTeam'] in o.text:
                                    match1['awayTeamUrl'] = o['href'].replace('/wiki/','')
                        except:
                            match1['homeTeamUrl'], match1['awayTeamUrl'] = None, None
                        match1['awayTeamCountry'] = r.find_all('td')[2].find('img')['alt'].strip()
                        match1['score'] = r.find_all('td')[3].text.strip().replace(' (aet)','').replace('(aet)','')

                        match2['homeTeam'] = match1['awayTeam']
                        match2['awayTeam'] = match1['homeTeam']
                        match2['homeTeamUrl'] = match1['awayTeamUrl']
                        match2['awayTeamUrl'] = match1['homeTeamUrl']
                        match2['homeTeamCountry'] = match1['awayTeamCountry']
                        match2['awayTeamCountry'] = match1['homeTeamCountry']
                        match2['score'] = r.find_all('td')[4].text.strip().replace(' (aet)','').replace('(aet)','').replace('(Report)','').replace('(Report 2)','').replace(' ','')
                        try:
                            match2['score'] = match2['score'].split('–')[1] + '–' + match2['score'].split('–')[0]
                        except:
                            print(r)
                    
                        match1['estimatedDate'], match2['estimatedDate'] = find_next_fdate(r), find_next_fdate(r)

                        zapasy.append(match1)
                        zapasy.append(match2)

                    except Exception as E:

                        print(E)

    with open(os.path.join('data_raw',filename), "w+", encoding="utf-8") as soubor:
        soubor.write(json.dumps(zapasy))
    
    return(zapasy)

In [145]:
scrape_matches('1978%E2%80%9379_European_Cup_Winners%27_Cup.html')

scraping 1978%E2%80%9379_European_Cup_Winners%27_Cup_2.json
'NoneType' object is not subscriptable


[{'competition': '1978%E2%80%9379 European Cup Winners%27 Cup',
  'phase': 'First round',
  'homeTeam': 'AZ Alkmaar',
  'homeTeamCountry': 'Netherlands',
  'awayTeam': 'Ipswich Town',
  'homeTeamUrl': 'AZ_Alkmaar',
  'awayTeamUrl': 'Ipswich_Town_F.C.',
  'awayTeamCountry': 'England',
  'score': '0–0 (Report)(Report 2)',
  'estimatedDate': '13 September 1978'},
 {'competition': '1978%E2%80%9379 European Cup Winners%27 Cup',
  'phase': 'First round',
  'homeTeam': 'Ipswich Town',
  'awayTeam': 'AZ Alkmaar',
  'homeTeamUrl': 'Ipswich_Town_F.C.',
  'awayTeamUrl': 'AZ_Alkmaar',
  'homeTeamCountry': 'England',
  'awayTeamCountry': 'Netherlands',
  'score': '2–0',
  'estimatedDate': '13 September 1978'},
 {'competition': '1978%E2%80%9379 European Cup Winners%27 Cup',
  'phase': 'First round',
  'homeTeam': 'Zagłębie Sosnowiec',
  'homeTeamCountry': 'Poland',
  'awayTeam': 'SSW Innsbruck',
  'homeTeamUrl': 'Zag%C5%82%C4%99bie_Sosnowiec',
  'awayTeamUrl': 'FC_Wacker_Innsbruck',
  'awayTeamCount

In [146]:
cwc = ['https://en.wikipedia.org/wiki/1960%E2%80%9361_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1961%E2%80%9362_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1962%E2%80%9363_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1963%E2%80%9364_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1964%E2%80%9365_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1965%E2%80%9366_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1966%E2%80%9367_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1967%E2%80%9368_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1968%E2%80%9369_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1969%E2%80%9370_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1970%E2%80%9371_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1971%E2%80%9372_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1972%E2%80%9373_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1973%E2%80%9374_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1974%E2%80%9375_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1975%E2%80%9376_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1976%E2%80%9377_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1977%E2%80%9378_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1978%E2%80%9379_European_Cup_Winners%27_Cup',
        'https://en.wikipedia.org/wiki/1979%E2%80%9380_European_Cup_Winners%27_Cup']

In [147]:
cwc = [c.split('/')[-1] for c in cwc]

In [148]:
cwc

['1960%E2%80%9361_European_Cup_Winners%27_Cup',
 '1961%E2%80%9362_European_Cup_Winners%27_Cup',
 '1962%E2%80%9363_European_Cup_Winners%27_Cup',
 '1963%E2%80%9364_European_Cup_Winners%27_Cup',
 '1964%E2%80%9365_European_Cup_Winners%27_Cup',
 '1965%E2%80%9366_European_Cup_Winners%27_Cup',
 '1966%E2%80%9367_European_Cup_Winners%27_Cup',
 '1967%E2%80%9368_European_Cup_Winners%27_Cup',
 '1968%E2%80%9369_European_Cup_Winners%27_Cup',
 '1969%E2%80%9370_European_Cup_Winners%27_Cup',
 '1970%E2%80%9371_European_Cup_Winners%27_Cup',
 '1971%E2%80%9372_European_Cup_Winners%27_Cup',
 '1972%E2%80%9373_European_Cup_Winners%27_Cup',
 '1973%E2%80%9374_European_Cup_Winners%27_Cup',
 '1974%E2%80%9375_European_Cup_Winners%27_Cup',
 '1975%E2%80%9376_European_Cup_Winners%27_Cup',
 '1976%E2%80%9377_European_Cup_Winners%27_Cup',
 '1977%E2%80%9378_European_Cup_Winners%27_Cup',
 '1978%E2%80%9379_European_Cup_Winners%27_Cup',
 '1979%E2%80%9380_European_Cup_Winners%27_Cup']

In [149]:
html_files_2 = []
for h in html_files:
    for c in cwc:
        if c in h:
            html_files_2.append(h)
html_files_2

['1960%E2%80%9361_European_Cup_Winners%27_Cup.html',
 '1961%E2%80%9362_European_Cup_Winners%27_Cup.html',
 '1962%E2%80%9363_European_Cup_Winners%27_Cup.html',
 '1963%E2%80%9364_European_Cup_Winners%27_Cup.html',
 '1964%E2%80%9365_European_Cup_Winners%27_Cup.html',
 '1965%E2%80%9366_European_Cup_Winners%27_Cup.html',
 '1966%E2%80%9367_European_Cup_Winners%27_Cup.html',
 '1967%E2%80%9368_European_Cup_Winners%27_Cup.html',
 '1968%E2%80%9369_European_Cup_Winners%27_Cup.html',
 '1969%E2%80%9370_European_Cup_Winners%27_Cup.html',
 '1970%E2%80%9371_European_Cup_Winners%27_Cup.html',
 '1971%E2%80%9372_European_Cup_Winners%27_Cup.html',
 '1972%E2%80%9373_European_Cup_Winners%27_Cup.html',
 '1973%E2%80%9374_European_Cup_Winners%27_Cup.html',
 '1974%E2%80%9375_European_Cup_Winners%27_Cup.html',
 '1975%E2%80%9376_European_Cup_Winners%27_Cup.html',
 '1976%E2%80%9377_European_Cup_Winners%27_Cup.html',
 '1977%E2%80%9378_European_Cup_Winners%27_Cup.html',
 '1978%E2%80%9379_European_Cup_Winners%27_Cup.

In [150]:
for h in html_files_2:
    scrape_matches(h)

scraping 1960%E2%80%9361_European_Cup_Winners%27_Cup_2.json
scraping 1961%E2%80%9362_European_Cup_Winners%27_Cup_2.json
scraping 1962%E2%80%9363_European_Cup_Winners%27_Cup_2.json
scraping 1963%E2%80%9364_European_Cup_Winners%27_Cup_2.json
scraping 1964%E2%80%9365_European_Cup_Winners%27_Cup_2.json
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
scraping 1965%E2%80%9366_European_Cup_Winners%27_Cup_2.json
<tr>
<td style="text-align: right;"><b><a href="/wiki/West_Ham_United_F.C." title="West Ham United F.C.">West Ham United</a></b> <span class="flagicon"><span class="mw-image-border" typeof="mw:File"><a href="/wiki/England" title="England"><img alt="England" class="mw-file-element" data-file-height="480" data-file-width="800" decoding="async" height="14" src="//upload.wikimedia.org/wikipedia/en/thumb/b/be/Flag_of_England.svg/23px-Flag_of_England.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/b/be/Flag_of_England.svg/35px-Flag_of_England.svg.png 